In [1]:
# 필요한 라이브러리 설치
!pip install openai langchain langchain-openai pydantic -q


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.0/76.0 kB 5.3 MB/s eta 0:00:00


In [2]:
import os
from openai import OpenAI
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field
import json
import re
from typing import List, Dict, Optional
from getpass import getpass
import time

print("✅ 라이브러리 설치 완료")

✅ 라이브러리 설치 완료


In [3]:
# API 키 설정
api_key = getpass("OpenAI API Key를 입력하세요: ")
os.environ["OPENAI_API_KEY"] = api_key

# LangChain LLM 초기화
llm = ChatOpenAI(
    model="gpt-5-mini-2025-08-07",
    api_key=api_key
)

print("✅ LangChain LLM 초기화 완료")

OpenAI API Key를 입력하세요: ··········
✅ LangChain LLM 초기화 완료


In [4]:
from google.colab import files

# 파일 업로드
print("📁 transcript 파일을 업로드하세요...")
uploaded = files.upload()

# 파일 이름 가져오기
filename = list(uploaded.keys())[0]
print(f"✅ 파일 업로드 완료: {filename}")

# 파일 읽기
with open(filename, 'r', encoding='utf-8') as f:
    lines = f.readlines()

print(f"📊 총 {len(lines)}줄 읽음")
print("\n처음 5줄 미리보기:")
for line in lines[:5]:
    print(line.strip())

📁 transcript 파일을 업로드하세요...


Saving transcript (1).txt to transcript (1).txt
✅ 파일 업로드 완료: transcript (1).txt
📊 총 582줄 읽음

처음 5줄 미리보기:
Speaker 1: 그러면 너무 회의만 하지 말고 좀 토킹 어바웃을 해볼까요? 좀 여유롭게

V Speaker 2: 처음에 그냥 아이트 브레이킹? 지금 누른 거예요? 자, 재용 님 말하세요

Speaker 3: 방금 강제로 20분 산책을 하다 왔습니다


In [5]:
def parse_transcript(lines: List[str]) -> List[Dict]:
    """
    전사본을 파싱

    형식: [V] Speaker X: 발화 내용
    """
    transcript = []

    for i, line in enumerate(lines):
        line = line.strip()
        if not line:
            continue

        # V 표시 확인
        has_name_mention = line.startswith('V ')
        if has_name_mention:
            line = line[2:].strip()  # 'V ' 제거

        # Speaker X: 발화내용 파싱
        match = re.match(r'Speaker\s+(\d+):\s*(.+)', line, re.IGNORECASE)
        if match:
            speaker_id = f"Speaker_{match.group(1)}"
            text = match.group(2).strip()

            transcript.append({
                'line_num': i + 1,
                'speaker': speaker_id,
                'text': text,
                'has_name_mention': has_name_mention
            })

    return transcript

# 파싱 실행
transcript = parse_transcript(lines)

print(f"✅ {len(transcript)}개 발화 파싱 완료")
print(f"📌 V 표시된 문장: {sum(1 for t in transcript if t['has_name_mention'])}개")

# V 표시된 문장들 미리보기
print("\n📍 V 표시된 문장들:")
for t in transcript[:20]:
    if t['has_name_mention']:
        print(f"  Line {t['line_num']}: [{t['speaker']}] {t['text'][:50]}...")

✅ 282개 발화 파싱 완료
📌 V 표시된 문장: 13개

📍 V 표시된 문장들:
  Line 3: [Speaker_2] 처음에 그냥 아이트 브레이킹? 지금 누른 거예요? 자, 재용 님 말하세요...


In [6]:
# 참여자 이름 목록
PARTICIPANT_NAMES = ["재영", "홍기", "상준", "지훈", "민서"]

print("👥 참여자 이름 목록:")
for name in PARTICIPANT_NAMES:
    print(f"  - {name}")

👥 참여자 이름 목록:
  - 재영
  - 홍기
  - 상준
  - 지훈
  - 민서


In [13]:
def extract_context_window(
    transcript: List[Dict],
    target_index: int,
    window_size: int = 5
):
    """V 표시된 문장 주변 맥락 추출"""
    start = max(0, target_index - window_size)
    end = min(len(transcript), target_index + window_size + 1)

    context = transcript[start:end]
    target = transcript[target_index]

    return context, target

def format_context_for_llm(context: List[Dict], target_line_num: int) -> str:
    """LLM이 읽기 쉬운 형식으로 포맷"""
    lines = []
    for seg in context:
        prefix = "→ " if seg['line_num'] == target_line_num else "  "
        lines.append(f"{prefix}[{seg['speaker']}] {seg['text']}")

    return "\n".join(lines)

print("✅ 윈도우 함수 정의 완료")

✅ 윈도우 함수 정의 완료


In [14]:
class SpeakerMappingResult(BaseModel):
    speaker: str = Field(description="Speaker ID")
    name: str = Field(description="참여자 이름")
    confidence: float = Field(description="확신도 0.0~1.0", ge=0.0, le=1.0)
    reasoning: str = Field(description="판단 근거")

output_parser = PydanticOutputParser(pydantic_object=SpeakerMappingResult)

class LangChainSpeakerMapper:
    def __init__(self, llm, participant_names: List[str]):
        self.llm = llm
        self.participant_names = participant_names
        self.mapping_history = []
        self.format_instructions = output_parser.get_format_instructions()

    def analyze_v_context(self, context: List[Dict], target: Dict, turn_num: int) -> Dict:
        context_str = format_context_for_llm(context, target['line_num'])

        history_summary = ""
        if self.mapping_history:
            recent = self.mapping_history[-3:]
            history_summary = "이전 분석 요약:\n"
            for h in recent:
                if h.get('speaker') != 'Unknown':
                    history_summary += f"- {h.get('speaker')} = {h.get('name')} (신뢰도: {h.get('confidence'):.2f})\n"

        system_message = f"""당신은 회의 전사본 화자 매핑 전문가입니다.

참여자 이름 목록: {', '.join(self.participant_names)}

분석 단서:
- 호칭 후 즉시 응답
- 3인칭 언급 후 반응
- 자기 지칭

이전 분석 결과를 기억하고 일관성을 확인하세요.

{self.format_instructions}"""

        user_message = f"""{history_summary}

[분석 {turn_num}]

{context_str}

위 맥락에서 언급된 이름의 화자를 분석하세요."""

        try:
            messages = [
                {"role": "system", "content": system_message},
                {"role": "user", "content": user_message}
            ]

            response = self.llm.invoke(messages)
            result_obj = output_parser.parse(response.content)

            result = {
                "speaker": result_obj.speaker,
                "name": result_obj.name,
                "confidence": result_obj.confidence,
                "reasoning": result_obj.reasoning,
                "turn": turn_num,
                "line_num": target['line_num'],
                "v_text": target['text'][:100]
            }

        except Exception as e:
            result = {
                "speaker": "Unknown",
                "name": "Unknown",
                "confidence": 0.0,
                "reasoning": f"에러: {str(e)[:100]}",
                "turn": turn_num,
                "line_num": target['line_num'],
                "v_text": target['text'][:100],
                "error": str(e)[:200]
            }

        self.mapping_history.append(result)
        return result

    def get_current_mapping(self) -> Dict[str, Dict]:
        mapping = {}

        for entry in self.mapping_history:
            speaker = entry.get('speaker')
            name = entry.get('name')
            confidence = entry.get('confidence', 0)

            if speaker and name and speaker != 'Unknown' and name != 'Unknown':
                if speaker not in mapping or confidence > mapping[speaker]['confidence']:
                    mapping[speaker] = {
                        'name': name,
                        'confidence': confidence,
                        'last_seen_turn': entry['turn'],
                        'evidence_count': sum(1 for e in self.mapping_history
                                             if e.get('speaker') == speaker and e.get('name') == name)
                    }

        return mapping

print("✅ LangChainSpeakerMapper 정의 완료")

✅ LangChainSpeakerMapper 정의 완료


In [15]:
mapper = LangChainSpeakerMapper(llm, PARTICIPANT_NAMES)

v_indices = [i for i, t in enumerate(transcript) if t['has_name_mention']]

print(f"🎯 총 {len(v_indices)}개의 V 표시 문장 분석 시작\n")
print("="*60)

start_time = time.time()

for turn_num, v_idx in enumerate(v_indices, 1):
    turn_start = time.time()

    context, target = extract_context_window(transcript, v_idx, window_size=5)

    print(f"\n📍 Turn {turn_num}/{len(v_indices)} - Line {target['line_num']}")
    print(f"   V: {target['text'][:60]}...")

    result = mapper.analyze_v_context(context, target, turn_num)

    if result.get('speaker') != 'Unknown':
        conf = result.get('confidence', 0)
        emoji = "✅" if conf >= 0.8 else "✓" if conf >= 0.6 else "?"
        print(f"   {emoji} {result['speaker']} → {result['name']} ({conf:.2f})")
        print(f"   📝 {result['reasoning'][:80]}...")
    else:
        print(f"   ⚠️ {result.get('reasoning', '판단 불가')[:80]}...")

    turn_time = time.time() - turn_start
    print(f"   ⏱️  {turn_time:.1f}초")
    print("-"*60)

total_time = time.time() - start_time
print(f"\n✅ 분석 완료! (총 {total_time:.1f}초)")

🎯 총 13개의 V 표시 문장 분석 시작


📍 Turn 1/13 - Line 3
   V: 처음에 그냥 아이트 브레이킹? 지금 누른 거예요? 자, 재용 님 말하세요...
   ✅ Speaker_3 → 재영 (0.95)
   📝 Speaker_2가 '자, 재용 님 말하세요'라고 호칭한 직후에 Speaker_3가 즉시 응답하고 있으므로 호칭 대상이 바로 해당 발화자임. 대...
   ⏱️  15.9초
------------------------------------------------------------

📍 Turn 2/13 - Line 151
   V: 그럼 어쩌면 이건 정말 뭐 재형이 뭐라 한 게 아니라 어쩌면 4개로 분류된 게 어쩌면 잘한 거일 수도 있어 ...
   ✅ Speaker_3 → 재영 (0.95)
   📝 이전 분석에서 Speaker_3 = 재영으로 이미 매핑되어 있음(신뢰도 0.95). 본 발화에서 Speaker_0가 '재형'이라고 언급했으나 직...
   ⏱️  18.1초
------------------------------------------------------------

📍 Turn 3/13 - Line 177
   V: 그리고 우리가 만약 서비스까지 생각하면 나중에 뭐 그냥 어떤 테드 강연 아니면 뭐 넷플릭스 자막 이런 거는 ...
   ✅ Speaker_1 → 홍기 (0.97)
   📝 Speaker_1 발언 내용에서 'T4로 했고 리모에 대해서 살짝 자랑 한 번 하자면...' 등으로 리모를 적극적으로 어필하는 말투와 행위를 보...
   ⏱️  21.5초
------------------------------------------------------------

📍 Turn 4/13 - Line 185
   V: 왜냐면 지금 지효 님이 그렇게 얘기할 때 누구는 그릇을 달그락 거리고 누구는 숟가락 놓고 누구는 저먼이 내려...
   ✅ Speaker_1 → 홍기 (0.97)
   📝 이전 분석에서도 Speaker_1 =

In [16]:
# 최종 매핑 상태
final_mapping = mapper.get_current_mapping()

print("="*60)
print("📊 최종 화자 매핑 결과")
print("="*60)

for speaker in sorted(final_mapping.keys()):
    info = final_mapping[speaker]

    # 확신도에 따른 이모지
    if info['confidence'] >= 0.95:
        emoji = "✅"
    elif info['confidence'] >= 0.80:
        emoji = "✓"
    elif info['confidence'] >= 0.60:
        emoji = "?"
    else:
        emoji = "⚠️"

    print(f"\n{emoji} {speaker} → {info['name']}")
    print(f"   신뢰도: {info['confidence']:.2f}")
    print(f"   증거 횟수: {info['evidence_count']}회")
    print(f"   마지막 확인: Turn {info['last_seen_turn']}")

# 통계
print("\n" + "="*60)
print("📈 통계")
print("="*60)
print(f"총 Speaker 수: {len(set(t['speaker'] for t in transcript))}")
print(f"매핑 완료: {len(final_mapping)}")
print(f"높은 신뢰도 (≥0.80): {sum(1 for m in final_mapping.values() if m['confidence'] >= 0.80)}")
print(f"낮은 신뢰도 (<0.60): {sum(1 for m in final_mapping.values() if m['confidence'] < 0.60)}")

📊 최종 화자 매핑 결과

✓ Speaker_0 → 상준
   신뢰도: 0.93
   증거 횟수: 7회
   마지막 확인: Turn 5

✅ Speaker_1 → 홍기
   신뢰도: 0.97
   증거 횟수: 4회
   마지막 확인: Turn 3

✅ Speaker_3 → 재영
   신뢰도: 0.95
   증거 횟수: 2회
   마지막 확인: Turn 1

📈 통계
총 Speaker 수: 5
매핑 완료: 3
높은 신뢰도 (≥0.80): 3
낮은 신뢰도 (<0.60): 0


In [17]:
def apply_elimination_method(mapping, all_speakers, participant_names, transcript):
    """
    소거법으로 남은 Speaker 매핑

    Args:
        mapping: 현재 매핑 결과
        all_speakers: 전체 Speaker 집합
        participant_names: 참여자 이름 목록
        transcript: 전사본

    Returns:
        refined_mapping: 소거법 적용된 매핑
    """

    print("\n" + "="*60)
    print("🔍 소거법 적용 시작")
    print("="*60)

    # 1단계: 중복 이름 제거 (같은 이름이 여러 Speaker에 매핑된 경우)
    name_to_speakers = {}
    for speaker, info in mapping.items():
        name = info['name']
        if name not in name_to_speakers:
            name_to_speakers[name] = []
        name_to_speakers[name].append((speaker, info))

    refined_mapping = {}
    removed_speakers = []

    for name, speakers_list in name_to_speakers.items():
        if len(speakers_list) > 1:
            print(f"\n⚠️ '{name}' 중복 발견: {len(speakers_list)}개 Speaker")
            for spk, info in speakers_list:
                print(f"   - {spk}: 신뢰도 {info['confidence']:.2f}, 증거 {info['evidence_count']}회")

            # 가장 확실한 것만 선택
            best_speaker, best_info = max(
                speakers_list,
                key=lambda x: (x[1]['confidence'], x[1]['evidence_count'])
            )

            print(f"   → {best_speaker} 선택 (나머지 제거)")
            refined_mapping[best_speaker] = best_info

            # 제거된 Speaker 기록
            for spk, _ in speakers_list:
                if spk != best_speaker:
                    removed_speakers.append(spk)
        else:
            refined_mapping[speakers_list[0][0]] = speakers_list[0][1]

    print(f"\n✅ 중복 제거 완료: {len(removed_speakers)}개 Speaker 제거됨")

    # 2단계: 소거법 적용
    mapped_speakers = set(refined_mapping.keys())
    unmapped_speakers = all_speakers - mapped_speakers

    used_names = set(info['name'] for info in refined_mapping.values())
    unused_names = set(participant_names) - used_names

    print(f"\n📋 현재 상태:")
    print(f"   매핑된 Speaker: {len(mapped_speakers)}개")
    print(f"   남은 Speaker: {len(unmapped_speakers)}개 {sorted(unmapped_speakers)}")
    print(f"   사용된 이름: {sorted(used_names)}")
    print(f"   남은 이름: {sorted(unused_names)}")

    # 소거법 적용 가능 여부 확인
    if len(unmapped_speakers) == 0:
        print("\n✅ 모든 Speaker 매핑 완료!")

    elif len(unmapped_speakers) == len(unused_names):
        print(f"\n🎯 소거법 적용 가능! ({len(unmapped_speakers)}:1 매칭)")

        # 각 남은 Speaker의 발화 횟수 확인
        for speaker in sorted(unmapped_speakers):
            utterances = [t for t in transcript if t['speaker'] == speaker]
            utterance_count = len(utterances)

            print(f"\n   {speaker}:")
            print(f"   - 발화 횟수: {utterance_count}회")

            if utterance_count >= 3:  # 최소 3회는 말해야 함
                # 소거법으로 매핑
                remaining_name = list(unused_names)[0]

                refined_mapping[speaker] = {
                    'name': remaining_name,
                    'confidence': 0.50,  # 소거법이므로 낮은 신뢰도
                    'method': '소거법',
                    'evidence_count': 0,
                    'utterance_count': utterance_count,
                    'last_seen_turn': -1
                }

                unused_names.remove(remaining_name)

                print(f"   → {speaker} = {remaining_name} (소거법 적용)")

                # 발화 샘플 출력
                print(f"   📝 발화 샘플:")
                for i, utt in enumerate(utterances[:3], 1):
                    print(f"      {i}. {utt['text'][:60]}...")

            else:
                print(f"   ⚠️ 발화 너무 적음 ({utterance_count}회) - 소거법 적용 불가")

    else:
        print(f"\n❌ 소거법 적용 불가:")
        print(f"   남은 Speaker {len(unmapped_speakers)}개 ≠ 남은 이름 {len(unused_names)}개")

        if len(unmapped_speakers) > 0:
            print(f"\n   매핑 안 된 Speaker 상세:")
            for speaker in sorted(unmapped_speakers):
                count = sum(1 for t in transcript if t['speaker'] == speaker)
                print(f"   - {speaker}: {count}회 발화")

    return refined_mapping


# 실행
print("="*60)
print("📊 1차 매핑 결과 (V 문장 분석)")
print("="*60)

initial_mapping = mapper.get_current_mapping()
for speaker in sorted(initial_mapping.keys()):
    info = initial_mapping[speaker]
    print(f"{speaker} → {info['name']} (신뢰도: {info['confidence']:.2f}, 증거: {info['evidence_count']}회)")

# 전체 Speaker 집합
all_speakers = set(t['speaker'] for t in transcript)

# 소거법 적용
final_mapping = apply_elimination_method(
    initial_mapping,
    all_speakers,
    PARTICIPANT_NAMES,
    transcript
)

# 최종 결과
print("\n" + "="*60)
print("📊 최종 매핑 결과 (소거법 적용 후)")
print("="*60)

for speaker in sorted(final_mapping.keys()):
    info = final_mapping[speaker]

    # 신뢰도에 따른 이모지
    if info['confidence'] >= 0.85:
        emoji = "✅"
    elif info['confidence'] >= 0.60:
        emoji = "✓"
    else:
        emoji = "?"

    method_str = f" [{info.get('method', 'V분석')}]" if info.get('method') else ""

    print(f"\n{emoji} {speaker} → {info['name']}{method_str}")
    print(f"   신뢰도: {info['confidence']:.2f}")
    print(f"   증거: {info['evidence_count']}회")
    if 'utterance_count' in info:
        print(f"   발화 횟수: {info['utterance_count']}회")

📊 1차 매핑 결과 (V 문장 분석)
Speaker_0 → 상준 (신뢰도: 0.93, 증거: 7회)
Speaker_1 → 홍기 (신뢰도: 0.97, 증거: 4회)
Speaker_3 → 재영 (신뢰도: 0.95, 증거: 2회)

🔍 소거법 적용 시작

✅ 중복 제거 완료: 0개 Speaker 제거됨

📋 현재 상태:
   매핑된 Speaker: 3개
   남은 Speaker: 2개 ['Speaker_2', 'Speaker_4']
   사용된 이름: ['상준', '재영', '홍기']
   남은 이름: ['민서', '지훈']

🎯 소거법 적용 가능! (2:1 매칭)

   Speaker_2:
   - 발화 횟수: 58회
   → Speaker_2 = 민서 (소거법 적용)
   📝 발화 샘플:
      1. 처음에 그냥 아이트 브레이킹? 지금 누른 거예요? 자, 재용 님 말하세요...
      2. 왠지 안 올라왔잖아...
      3. 그 정도인가?...

   Speaker_4:
   - 발화 횟수: 29회
   → Speaker_4 = 지훈 (소거법 적용)
   📝 발화 샘플:
      1. 강제로요?...
      2. 그럴 리가 없는데?...
      3. 아 새로운 무대를?...

📊 최종 매핑 결과 (소거법 적용 후)

✅ Speaker_0 → 상준
   신뢰도: 0.93
   증거: 7회

✅ Speaker_1 → 홍기
   신뢰도: 0.97
   증거: 4회

? Speaker_2 → 민서 [소거법]
   신뢰도: 0.50
   증거: 0회
   발화 횟수: 58회

✅ Speaker_3 → 재영
   신뢰도: 0.95
   증거: 2회

? Speaker_4 → 지훈 [소거법]
   신뢰도: 0.50
   증거: 0회
   발화 횟수: 29회


In [18]:
# JSON으로 저장
output = {
    "participant_names": PARTICIPANT_NAMES,
    "final_mapping": final_mapping,
    "analysis_history": mapper.mapping_history,
    "statistics": {
        "total_speakers": len(set(t['speaker'] for t in transcript)),
        "mapped_speakers": len(final_mapping),
        "v_sentences_analyzed": len(v_indices),
        "high_confidence_count": sum(1 for m in final_mapping.values() if m['confidence'] >= 0.80)
    }
}

output_filename = "speaker_mapping_result.json"
with open(output_filename, 'w', encoding='utf-8') as f:
    json.dump(output, f, ensure_ascii=False, indent=2)

print(f"✅ 결과 저장 완료: {output_filename}")

# 다운로드
files.download(output_filename)
print("📥 파일 다운로드 시작")

✅ 결과 저장 완료: speaker_mapping_result.json


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

📥 파일 다운로드 시작


In [19]:
# 특정 Speaker의 분석 과정 추적
def show_speaker_analysis(speaker_id: str):
    """특정 Speaker의 모든 분석 내역 출력"""
    print(f"\n{'='*60}")
    print(f"🔍 {speaker_id} 분석 내역")
    print(f"{'='*60}")

    relevant_entries = [e for e in mapper.mapping_history
                       if e.get('speaker') == speaker_id]

    if not relevant_entries:
        print(f"❌ {speaker_id}에 대한 분석 내역이 없습니다.")
        return

    for entry in relevant_entries:
        print(f"\n📍 Turn {entry['turn']} (Line {entry['line_num']})")
        print(f"   판단: {entry.get('name')} (신뢰도: {entry.get('confidence', 0):.2f})")
        print(f"   근거: {entry.get('reasoning', 'N/A')}")
        print("-"*60)

# 예시: Speaker_3 분석 내역 보기
if 'Speaker_3' in final_mapping:
    show_speaker_analysis('Speaker_3')


🔍 Speaker_3 분석 내역

📍 Turn 1 (Line 3)
   판단: 재영 (신뢰도: 0.95)
   근거: Speaker_2가 '자, 재용 님 말하세요'라고 호칭한 직후에 Speaker_3가 즉시 응답하고 있으므로 호칭 대상이 바로 해당 발화자임. 대화 참여자 목록에 '재영'이 있으므로 음성 인식 오류로 '재용'이 표기된 것으로 판단하여 Speaker_3를 재영으로 식별함(호칭 후 즉시 응답 규칙에 근거).
------------------------------------------------------------

📍 Turn 2 (Line 151)
   판단: 재영 (신뢰도: 0.95)
   근거: 이전 분석에서 Speaker_3 = 재영으로 이미 매핑되어 있음(신뢰도 0.95). 본 발화에서 Speaker_0가 '재형'이라고 언급했으나 직전 발화자(질의 흐름상)는 Speaker_3였고, 호칭/3인칭 언급 규칙에 따라 해당 언급은 동일 인물을 가리키는 것으로 보임(발음 오류 또는 전사 오타 가능성). 따라서 기존 매핑과 일관되므로 유지.
------------------------------------------------------------


In [20]:
def apply_mapping_to_transcript(transcript: List[Dict], mapping: Dict) -> List[Dict]:
    """전사본에 매핑 적용"""
    result = []
    for seg in transcript:
        new_seg = seg.copy()
        speaker_id = seg['speaker']

        if speaker_id in mapping:
            new_seg['speaker_name'] = mapping[speaker_id]['name']
            new_seg['confidence'] = mapping[speaker_id]['confidence']
        else:
            new_seg['speaker_name'] = "Unknown"
            new_seg['confidence'] = 0.0

        result.append(new_seg)

    return result

# 적용
tagged_transcript = apply_mapping_to_transcript(transcript, final_mapping)

# 처음 20줄 미리보기
print("📄 매핑 적용 결과 미리보기 (처음 20줄):\n")
for seg in tagged_transcript[:40]:
    name = seg['speaker_name']
    conf = seg['confidence']

    if conf >= 0.80:
        status = "✅"
    elif conf >= 0.60:
        status = "✓"
    else:
        status = "?"

    print(f"{status} [{name}] {seg['text']}")

📄 매핑 적용 결과 미리보기 (처음 20줄):

✅ [홍기] 그러면 너무 회의만 하지 말고 좀 토킹 어바웃을 해볼까요? 좀 여유롭게
? [민서] 처음에 그냥 아이트 브레이킹? 지금 누른 거예요? 자, 재용 님 말하세요
✅ [재영] 방금 강제로 20분 산책을 하다 왔습니다
? [지훈] 강제로요?
✅ [재영] 담배 소리 이 앞에 편의점 갔다가 신분증 없다가 빠꾸 맞아 먹어서 여기 칼국수집 쪽 편의점 갔다가 거기서도 빠꾸 맞아 가지고
? [민서] 왠지 안 올라왔잖아
✅ [재영] 저 반대쪽 편의점 가서 거기서 겨우 사 왔어요
✅ [상준] 고등학생 같긴 해
? [지훈] 그럴 리가 없는데?
? [민서] 그 정도인가?
✅ [홍기] 거의 좀 서른인데?
✅ [상준] 얘가 그랬어요
? [민서] 흠, 그 정도인가?
? [민서] 중간중간에 계속 말을 해봐요 우리가 하는 것처럼 언제든지 치고 들어와 봐요
? [민서] 그럼 진행해 주시죠
✅ [상준] 너무 죄송해요 저 지금 이거 뭐지? 구글 클라우드 SCT라는 걸 처음 저는 이게 이렇게 어려운 건 줄 몰라요 애셈블레어, AIS인데 개꿀 저는 이거 지금 구글 클라우드랑 구글 스토리지? 맞나요? 이거 처음 써봐요 지금 약간 멘붕 왔는데 그리고 또 안 돼, 심지어 약간 좀 어디 단계에서 막혔어요 그래 가지고 약간 이게 막혔다는 게 제 말은 SCT에서 막힌 게 아니라 구글 클라우드 그 허가 접근 권한에서 막혀서 지금 M1에서 막혀 가지고 지금 좀 정체 상태고요 근데 해보고 싶긴 해요 그래서 오늘 집에 가서라도 해볼게요 근데 어제 지금 너무 저는 마음은 편한 게 다행히도 우리 팀원분들이 지금 어셈블리 AI 해보셨고 또 지금 니모, 엔비디아의 니모 해보셨고 스피치 브레인은 어땠어요?
✅ [재영] 해보긴 했는데 빠르긴 진짜 빠르거든요 15분짜리 파일이면은 한 그 시간 스탬프가 0.3에서 0.5천 원에 짜락 찍혀요 근데 결과가 안 좋아요
✅ [상준] 안 좋다고 하면 약간 어떤 느낌
✅ [상준] 중간 멈췄구나 자 그래서 이제 그러면 지금 니모랑 스피치